In [ ]:
def data_search (dataset, latest_version, temporal, bounding_box):
    import requests
    import json
    from statistics import mean
    
    # Query number of granules (paging over results)
    granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
    params = {
        'short_name': dataset,
        'version': latest_version,
        'bounding_box': bounding_box,
        'temporal': temporal,
        'page_size': 100,
        'page_num': 1,
    }
    granules = []
    headers={'Accept': 'application/json'}
    while True:
        response = requests.get(granule_search_url, params=params, headers=headers)
        results = json.loads(response.content)

        if len(results['feed']['entry']) == 0:
            # Out of results, so break out of loop
            break

        # Collect results and increment page_num
        granules.extend(results['feed']['entry'])
        params['page_num'] += 1

    print('There are', len(granules), 'granules of', dataset, 'version', latest_version, 'over my area and time of interest.')
    granule_sizes = [float(granule['granule_size']) for granule in granules]
    print(f'The average size of each granule is {mean(granule_sizes):.2f} MB and the total size of all {len(granules)} granules is {sum(granule_sizes):.2f} MB')
    return;

In [ ]:
def service_search (dataset, latest_version, bounding_box, temporal, uid, pswd):
    import requests

    # Query service capability URL 
    from xml.etree import ElementTree as ET
    capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{dataset}.{latest_version}.xml'

    # Create session to store cookie and pass credentials to capabilities url
    session = requests.session()
    s = session.get(capability_url)
    response = session.get(s.url,auth=(uid,pswd))
    root = ET.fromstring(response.content)

    #collect lists with each service option
    subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

    # variable subsetting
    variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
    variables_raw = [variables[i]['value'] for i in range(len(variables))]
    variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
    variable_vals = [v.replace(':', '/') for v in variables_join]

    # reformatting
    formats = [Format.attrib for Format in root.iter('Format')]
    format_vals = [formats[i]['value'] for i in range(len(formats))]
    format_vals.remove('')

    # reformatting options that support reprojection
    normalproj = [Projections.attrib for Projections in root.iter('Projections')]
    normalproj_vals = []
    normalproj_vals.append(normalproj[0]['normalProj'])
    format_proj = normalproj_vals[0].split(',')
    format_proj.remove('')
    format_proj.append('No reformatting')

    # reprojection options
    projections = [Projection.attrib for Projection in root.iter('Projection')]
    proj_vals = []
    for i in range(len(projections)):
        if (projections[i]['value']) != 'NO_CHANGE' :
            proj_vals.append(projections[i]['value'])

    # reformatting options that do not support reprojection
    no_proj = [i for i in format_vals if i not in format_proj]

    # SMAP-specific reprojection logic

    #L1-L2 reprojection/reformatting options
    if dataset == 'SPL1CTB' or 'SPL1CTB_E' or 'SPL2SMA' or 'SPL2SMP' or 'SPL2SMP_E' or 'SPL2SMAP': 
        format_proj = ['GeoTIFF', 'NetCDF4-CF', 'HDF-EOS5']
        no_proj = [i for i in format_vals if i not in format_proj]
    elif dataset == 'SPL2SMAP_S' or 'SPL3SMA' or 'SPL3SMP' or 'SPL3SMP_E' or 'SPL3SMAP' or 'SPL3FTA' or 'SPL3FTP' or 'SPL3FTP_E' or 'SPL4SMAU' or 'SPL4SMGP' or 'SPL4SMLM' or 'SPL4CMDL': 
        format_proj = ['No reformatting', 'GeoTIFF', 'NetCDF4-CF', 'HDF-EOS5']
        no_proj = [i for i in format_vals if i not in format_proj]

    #print service information depending on service availability and select service options
    if len(subagent) < 1 :
        print('No services exist for', dataset, 'version', latest_version)
        agent = 'NO'
        bbox = ''
        time_var = ''
        reformat = ''
        projection = ''
        projection_parameters = ''
        coverage = ''
    else:
        agent = ''
        subdict = subagent[0]
        if subdict['spatialSubsetting'] == 'true':
            ss = input('Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n)')
            if ss == 'y': bbox = bounding_box
            else: bbox = ''
        if subdict['temporalSubsetting'] == 'true':
            ts = input('Subsetting by time, based on the temporal range inputted above, is available. Would you like to request this service? (y/n)')
            if ts == 'y': time_var = temporal 
            else: time_var = ''
        else: time_var = ''
        if len(format_vals) > 0 :
            print('These reformatting options are available:', format_vals)
            reformat = input('If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank.')
            # select reprojection options based on reformatting selection
            if reformat in format_proj and len(proj_vals) > 0 : 
                print('These reprojection options are available with your requested format:', proj_vals)
                projection = input('If you would like to reproject, copy and paste the reprojection option you would like (make sure to omit quotes, e.g. GEOGRAPHIC), otherwise leave blank.')
                # Enter required parameters for UTM North and South
                if projection == 'UTM NORTHERN HEMISPHERE' or projection == 'UTM SOUTHERN HEMISPHERE': 
                    NZone = input('Please enter a UTM zone (1 to 60 for Northern Hemisphere; -60 to -1 for Southern Hemisphere):')
                    projection_parameters = str('NZone:' + NZone)
                else: projection_parameters = ''
            else: 
                print('No reprojection options are supported with your requested format')
                projection = ''
                projection_parameters = ''
        else: 
            reformat = ''
            projection = ''
            projection_parameters = ''
            
    return;